# 使用 qianfan sdk 构建本地评估模型



千帆大模型平台提供了在线进行自动评估的方式，而有时我们希望有更灵活的自动评估方式。

本文将以评估文本摘要效果为例子来介绍

1. 如何使用千帆 sdk 封装自定义本地评估方法
2. 使用函数封装手搓其原理实现。

以下是本文封装部分使用的库与作用
    Dataset：提供数据集接口，用于构造待评估数据集，方便在本地、平台、huggingface多端拉取数据集。
    Prompt：提供数据模板，用于指导模型进行评估，可联网获取prompt或者使用prompt优化功能。
    Service：提供模型服务接口，在evaluateManager中为没有生成答案的数据集提供生成能力。
    EvaluateManager：提供评估任务管理功能，封装了异步调用算子，可以方便地实现多线程评估。

此处使用0.3.0版本以上的qianfan sdk。

In [ ]:
%pip install -U "qianfan>=0.3.0"

进行鉴权认证。

In [2]:
import os

os.environ['QIANFAN_ACCESS_KEY'] = 'your_access_key'
os.environ['QIANFAN_SECRET_KEY'] = 'your_secret_key'

# 使用 Service 对象进行评估时，请按实际情况填写 QPS LIMIT，
# 取值为所有 Service QPS Limit中的最小值
os.environ["QIANFAN_QPS_LIMIT"] = "1"
os.environ['QIANFAN_LLM_API_RETRY_COUNT'] = "3"

## 数据与模板准备

首先构造待评估数据集，此处用本地数据集，也可以拉取千帆平台上的数据集

In [3]:
from qianfan.dataset import Dataset

ds = Dataset.load(data_file="data_summerize/excerpt.jsonl", organize_data_as_group=True, input_columns=["prompt"], reference_column="response")

print(ds[0])

{'prompt': '新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。', 'response': [['修改后的立法法全文公布']], '_group': 0}


然后创建prompt模版，用于引导模型进行评估。

由于摘要任务没有标准答案，所以此处模板不提供，如果是评估问答任务，则需要提供标准答案以评估正确率。

In [4]:
from qianfan.common import Prompt

evaluation_prompt_template = """
你是一名裁判员，负责为给定新闻的摘要评分。

评价标准：

{criteria}

请你遵照以下的评分步骤：
{steps}


例子：

新闻：

{prompt}

摘要：

{response}


根据答案的综合水平给出0到{max_score}之间的整数评分。
如果答案存在明显的不合理之处，则应给出一个较低的评分。
如果答案符合以上要求并且与参考答案含义相似，则应给出一个较高的评分

你的回答模版只能输出整数评分,不输出文字
"""

evaluation_prompt = Prompt(
    name="evaluation_prompt",
    template=evaluation_prompt_template
)

接着指定评估指标和评估步骤。

此处从四个维度进行评估，分别是相关性、连贯性、一致性、流畅度

相关性

In [5]:
relevance_metric = """
相关性 - 摘要中重要内容的选择。 \ 
摘要只应包含来自源文档的重要信息。 \
惩罚包含冗余和多余信息的摘要。
"""

relevance_steps = """
1. 仔细阅读摘要和源文档。
2. 将摘要与源文档进行比较，并识别文章的主要观点。
3. 评估摘要是否涵盖了文章的主要观点，以及它包含多少无关或冗余信息。
"""

relevance_max_score = 10

连贯性

In [6]:

coherence_metric = """
连贯性 - 所有句子的总体质量。 \
我们将此维度与 DUC 质量问题结构性和连贯性相关， \
其中“摘要应具有良好的结构和组织，不应只是相关信息的堆叠，而应基于从句子到主题有关的信息的连贯性主体进行构建。” \

"""

coherence_steps = """
1. 仔细阅读文章并识别主要主题和关键点。
2. 阅读摘要并与文章进行比较。检查摘要是否涵盖了文章的主要主题和关键点，并以清晰且逻辑的顺序呈现它们
"""

coherence_max_score = 10

一致性

In [7]:
consistency_metric = """
一致性 - 摘要与摘要源的客观对齐。 \
客观一致的摘要只包含与源文档支持的陈述。 \
惩罚包含幻觉事实的摘要。
"""

consistency_steps = """
1. 仔细阅读文章并识别主要事实和细节。
2. 阅读摘要并与文章进行比较。检查摘要是否包含任何不支持的文章的事实错误。
3. 基于评估标准，为一致性分配分数。
"""

consistency_max_score = 10

流畅度

In [8]:
fluency_metric = """
流畅度：摘要的语法、拼写、标点、单词选择和句子结构的质量。
1：差。摘要有很多错误，使它难以理解或听起来不自然。
2：中。摘要有一些影响文本清晰度或流畅性的错误，但要点仍然可理解。
3：好。摘要很少或没有错误，易于阅读和遵循。
"""

fluency_steps = """
读取摘要并基于给定的标准评估其流畅度。从 1 到 3 分配一个流畅度分数。
"""

fluency_max_score = 3

In [9]:
evaluation_metrics = {
    "Relevance": (relevance_metric, relevance_metric, relevance_max_score),
    "Coherence": (coherence_metric, coherence_steps, coherence_max_score),
    "Consistency": (consistency_metric, consistency_steps, consistency_max_score),
    "Fluency": (fluency_metric, fluency_steps, fluency_max_score),
}

通过千帆sdk，我们有两种方法实现

## 方法一：封装本地评估器

使用千帆平台提供的本地评估工具类进行评估
此方法封装了异步请求，可以根据数据量并发进行评估

首先继承LocalEvaluator类，并实现evaluate方法

In [10]:
from typing import Any, Dict, List, Union
from qianfan.utils.pydantic import Field
from qianfan.evaluation.consts import (
    QianfanRefereeEvaluatorDefaultMaxScore,
    QianfanRefereeEvaluatorDefaultMetrics,
    QianfanRefereeEvaluatorDefaultSteps,
)
from qianfan.evaluation.evaluator import LocalEvaluator

class QianfanLocalEvaluator(LocalEvaluator):

    model: Any
    metric_name: str = Field(default="")
    prompt_metrics: str = Field(default=QianfanRefereeEvaluatorDefaultMetrics)
    prompt_steps: str = Field(default=QianfanRefereeEvaluatorDefaultSteps)
    prompt_max_score: int = Field(default=QianfanRefereeEvaluatorDefaultMaxScore)

    # evaluate三个参数，input是输入，reference是参考答案，output是模型输出的结果。
    # 当evaluationManager调用eval时，如果is_chat_service=True，则input为对话列表 List[Dict[str, Any]]，否则为文本 str。
    # 在该任务中，input是新闻，reference是摘要，output是service生成的摘要
    # 由于我们只是对新闻摘要进行评分，因此只需要用到input和reference
    def evaluate(
            self, input: Union[str, List[Dict[str, Any]]], reference: str, output: str
        ) -> Dict[str, Any]:
        # 将input调整为合适格式
        input_content = input if isinstance(input, str) else input[0].get('content','')
        # 生成评价模板
        prompt, _ = evaluation_prompt.render(
            criteria=self.prompt_metrics,
            steps=self.prompt_steps,
            prompt=input_content,
            response=reference,
            max_score=str(self.prompt_max_score),
            metric_name=self.metric_name,
        )
        # 调用模型获得评分
        msg = qianfan.Messages()
        msg.append(prompt)
        resp = self.model.do(
            messages=msg,
            temperature=0.1,
            top_p=1,
        )
        # print(f'{self.metric_name}|{input[0]}|{output}|{resp["result"].strip()}')
        return {self.metric_name: resp['result'].strip()}


In [11]:
import qianfan
from qianfan.model import Service

eb_turbo_service = Service(model="ERNIE-Bot-turbo")  # 加载生成回答的服务
chat_comp = qianfan.ChatCompletion(model="ERNIE-Bot-4")  # 实例化用于裁判的模型

[WARNING] [01-25 15:41:11] model.py:375 [t:8678109824]: service type should be specified before exec


附加本地评估器，供EvaluationManager调用

In [12]:
local_evaluators = []
for eval_type, (criteria, steps, max_score) in evaluation_metrics.items():
    local_evaluator = QianfanLocalEvaluator(
        prompt_metrics=criteria,
        prompt_steps=steps,
        prompt_max_score=max_score,
        model=chat_comp,
        metric_name=eval_type
    )
    local_evaluators.append(local_evaluator)

In [13]:
from qianfan.evaluation import EvaluationManager

em = EvaluationManager(local_evaluators=local_evaluators)
result = em.eval(
    [eb_turbo_service], ds,
    #  is_chat_service=False # 默认为True，此时input为List[Dict[str, Any]]，否则input为str,
)

In [14]:
result_dataset = result.result_dataset
result_df = {"News id":[],"Evaluation Type":[],"Score":[]}
for i, resp in enumerate(result_dataset.list()):
    for metric_name in evaluation_metrics.keys():
        result_df["News id"].append(i)
        # 添加评估类型及对应得分，因为此处只用了一个大模型，所以只取第一个结果
        result_df["Evaluation Type"].append(metric_name)
        result_df["Score"].append(resp["model_content"][0][metric_name])
    print(resp)

{'input_chats': [{'content': '新华社受权于18日全文播发修改后的《中华人民共和国立法法》，修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。', 'role': 'user'}], 'expected_output': '修改后的立法法全文公布', 'model_content': [{'Coherence': '8', 'Consistency': '10', 'Fluency': '3', 'Relevance': '8', 'content': '是的，您说的是正确的。新华社于2023年6月18日受权播发了修改后的《中华人民共和国立法法》。这次修改后的立法法分为“总则”“法律”“行政法规”“地方性法规、自治条例和单行条例、规章”“适用与备案审查”“附则”等6章，共计105条。\n\n这次修改后的立法法的发布，是为了更好地规范立法活动，加强宪法实施，推进科学立法、民主立法、依法立法，提高立法质量和效率，不断完善以宪法为核心的中国特色社会主义法律体系。\n\n同时，这次修改后的立法法也加强了对国家立法和地方立法的指导，明确了立法原则、立法程序、法律解释、备案审查等方面的规定，为各级人大及其常委会依法行使立法权提供了更加明确的法律依据和程序保障。\n\n总之，这次修改后的立法法的发布，对于推进全面依法治国、建设社会主义法治国家具有重要的意义。', 'llm_tag': 'None_None_ERNIE-Bot-turbo'}]}
{'input_chats': [{'content': '一辆小轿车，一名女司机，竟造成9死24伤。日前，深圳市交警局对事故进行通报：从目前证据看，事故系司机超速行驶且操作不当导致。目前24名伤员已有6名治愈出院，其余正接受治疗，预计事故赔偿费或超一千万元。', 'role': 'user'}], 'expected_output': '深圳机场9死24伤续：司机全责赔偿或超千万', 'model_content': [{'Coherence': '8', 'Consistency': '8', 'Fluency': '2', 'Relevance': '8', 'content': '这起事故是非常悲惨的，对受害者和家人造成

In [15]:
import pandas as pd
pivot_df = pd.DataFrame(result_df, index=None).pivot(
    columns="Evaluation Type", index="News id", values="Score"
).astype(int)
pivot_df['Mean_Score'] = pivot_df.mean(axis=1)
display(pivot_df)

Evaluation Type,Coherence,Consistency,Fluency,Relevance,Mean_Score
News id,,,,,
0,8,10,3,8,7.25
1,8,8,2,8,6.50
2,5,3,2,5,3.75


## 方法二：封装评估函数

除了上述进行本地评估的方法，也可以用ChatCompletion模型构造评估函数，从头开始制作评估的流程

In [16]:
import qianfan

def get_geval_score(chat_comp, evaluation_prompt, **kwargs):
    prompt, _ = evaluation_prompt.render(**kwargs)
    msg = qianfan.Messages()
    msg.append(prompt, role='user')
    resp = chat_comp.do(
        messages=msg,
        temperature=0.1,
        top_p=1,
    )
    return resp['result']

最后遍历数据集进行评估

In [17]:
chat_comp = qianfan.ChatCompletion(model="ERNIE-Bot-4")
result = {"Evaluation Type": [], "News id": [], "Score": []}
for eval_type, (criteria, steps, max_score) in evaluation_metrics.items():
    for ind, data in enumerate(ds):
        result["Evaluation Type"].append(eval_type)
        result["News id"].append(ind)
        evaluate = get_geval_score(
            chat_comp,
            evaluation_prompt,
            criteria=criteria,
            steps=steps,
            prompt=data['prompt'],
            response=data['response'][0][0],
            max_score=str(max_score),
            metric_name=eval_type
        )
        score = int(evaluate.strip())
        result["Score"].append(evaluate)


查看最终结果

In [18]:
pivot_df = pd.DataFrame(result, index=None).pivot(
    columns="Evaluation Type", index="News id", values="Score"
).astype(int)
pivot_df['Mean_Score'] = pivot_df.mean(axis=1)
display(pivot_df)

Evaluation Type,Coherence,Consistency,Fluency,Relevance,Mean_Score
News id,,,,,
0,8,10,3,9,7.50
1,8,8,2,8,6.50
2,5,3,2,5,3.75
